In [1]:
import os

In [2]:
try:
  from google.colab import drive
  drive.mount('/content/gdrive')

  PATH_DRIVER = '/content/gdrive/My Drive/'
  DATA_FOLDER = 'AI Dataset Sample/'

  !pip install fastai pydicom kornia --upgrade --quiet
  
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  os.environ['GITHUB_AUTH'] = user + ':' + password
  !git clone https://$GITHUB_AUTH@github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

  IN_COLAB = True
except:
  PATH_DRIVER = ''
  DATA_FOLDER = 'data/'
  IN_COLAB = False

from fastai.basics import *
from fastai.medical.imaging import *

In [3]:
PATH_PREFIX = os.path.join(PATH_DRIVER, DATA_FOLDER, '')
raw_folder = PATH_PREFIX + 'raw'
raw_folder = 'F:\Descargas\DICOMS'
organize_folder = PATH_PREFIX + 'organized'
preprocess_folder = PATH_PREFIX + 'preprocess'

In [5]:
from utils import organize_folders, generate_template, rename_patient

groups = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']
subgroup_length = 20
copy = True

In [10]:
organize_folders(raw_folder, organize_folder, groups=groups, subgroup_length=subgroup_length, force_extension='.dcm', copy=copy, debug=False)

In [11]:
dicom_files = get_files(organize_folder, extensions='.dcm')
rename_patient(dicom_files)

In [12]:
generate_template(organize_folder, groups, subgroup_length, excel=True)